In [1]:
import sys
sys.path.insert(1,"src/") # path to the folder with the code; alternatively, you can move the jupyter notebook to the folder src
import desc as ds

import tabulate

# Cohomology of wedge sum of spheres $S^2 \vee S^1$

We triangulate $S^2$ as a diamond on vertices $\{1,2,3,4,5\}$, and $S^1$ as a triangle $\{4,5,6\}$ (they share the vertex $4$). We call the face poset of this simplicial complex $\Sigma$. We compute the injective resolution of a constant sheaf on $\Sigma$, and then we compute the hypercohomology.

In [5]:
# We first definte the simplicial complex by giving a list of maximal simplices
sigma = ds.SimplicialComplex([
        (0,1,3), (0,1,4), (0,3,4),
        (1,2,3), (1,2,4), (2,3,4),
        (4,5), (4,6), (5,6)
])

# Create cochain complex, the injective resolution of the constant sheaf on Sigma
cplx_sigma = ds.ChainComplex.injective_resolution_of_constant_sheaf(sigma)
print("The hypercohomology is:", cplx_sigma.str_hypercohomology())

# The method hypercohomology returns a dictionary { degree : betti number }.
# The method str_hypercohomology we used above returns an easy-to-read string.
cplx_sigma.hypercohomology()

The hypercohomology is: 1,1,1


{0: 1, 1: 1, 2: 1}

In [6]:
# To explore the injective resolution a bit more, we can print the (co)chain.
# The notation is element^power, which says that in the given degree d,
# there is power-many indecomposable injectives [element] in the decomposition of I^d.
print("Injective resolution of k_Sigma:")
print(cplx_sigma.str_chain())

Injective resolution of k_Sigma:
234^1 + 124^1 + 123^1 + 034^1 + 014^1 + 013^1 + 56^1 + 46^1 + 45^1 --0--> 34^1 + 24^1 + 23^1 + 14^1 + 13^1 + 12^1 + 04^1 + 03^1 + 01^1 + 6^1 + 5^1 + 4^2 --1--> 4^1 + 3^1 + 2^1 + 1^1 + 0^1


In [10]:
# For the full information, we can print all the labeled matrices.
for degree, labeled_matrix in sorted(cplx_sigma.matrices.items()):
    labeled_matrix.print_matrix(title=f"eta^{degree}", form="short", zero_symbol="-")
    print()

  eta^-1
--------
     234
     124
     123
     034
     014
     013
      56
      46
      45

  eta^0  234    124    123    034    014    013    56    46    45
-------  -----  -----  -----  -----  -----  -----  ----  ----  ----
     34  1      -      -      1      -      -      -     -     -
     24  1      1      -      -      -      -      -     -     -
     23  1      -      1      -      -      -      -     -     -
     14  -      1      -      -      1      -      -     -     -
     13  -      -      1      -      -      1      -     -     -
     12  -      1      1      -      -      -      -     -     -
     04  -      -      -      1      1      -      -     -     -
     03  -      -      -      1      -      1      -     -     -
     01  -      -      -      -      1      1      -     -     -
      6  -      -      -      -      -      -      1     1     -
      5  -      -      -      -      -      -      1     -     1
      4  1      -      -      -      -      -      

# The example from the paper (Section 7)

We compute three different complexes of sheaves, each obtained via a triangulated (simplicial) map, $g$, $h$, and $l$, from $S^2 \vee S^1$ and $D^2$ to $S^2$.

Both maps $g$ and $h$ send $S^2$ identically to $S^2$. The first map, $g$, sends the circle to the point where it touches the sphere, and $h$ sends it to the equator. The third map we consider, $l$, from the closed disk to the sphere, is obtained by identifying all points along the boundary of the closed disk. See Figure 5 in the arXiv version of the paper (link in README).

In [2]:
# We first define the posets as simplicial complexes by listing the maximal simplices.
sigma = ds.SimplicialComplex([
        (0,1,3), (0,1,4), (0,3,4),
        (1,2,3), (1,2,4), (2,3,4),
        (4,5), (4,6), (5,6)
])
lambd = ds.SimplicialComplex([
            (0,1,3), (0,1,4), (0,3,4),
            (1,2,3), (1,2,4), (2,3,4)
])
disk  = ds.SimplicialComplex([
        (0,1,3), (0,3,4),
        (1,2,3), (1,2,4), (2,3,4),
        (1,4,5), (0,1,5), (0,5,6), (0,4,6)
])

# We define the maps between simplicial complexes by describing how the vertices are mapped.
# Vertices not explicitely given are sent identically, e.g., the first map, g,
# sends vertices as follows: {1:1, 2:2, 3:3, 4:4, 5:4, 6:4}
map_g  = ds.SimplicialMap(sigma, lambd, {5:4, 6:4})
map_h  = ds.SimplicialMap(sigma, lambd, {5:1, 6:3})
map_l  = ds.SimplicialMap(disk,  lambd, {5:4, 6:4})

# We compute injective resolutions of the constant sheaves on Sigma and the disk,
# and push the (co)chains forward via the maps g,h,l.
cplx_sigma  = ds.ChainComplex.injective_resolution_of_constant_sheaf(sigma)
cplx_g_push = cplx_sigma.pushforward(map_g).minimize()
cplx_h_push = cplx_sigma.pushforward(map_h).minimize()

cplx_disk   = ds.ChainComplex.injective_resolution_of_constant_sheaf(disk)
cplx_l_push = cplx_disk.pushforward(map_l).minimize()

# Minimization of the complexes is currently not done by default, because we use
# the alternative minimization process described in Section 5.6.3 of the paper:
# we compute the pullback via the identity map. This procedure could unnecessarily
# slow down computations with larger examples.

In [ ]:
# We can print the chains
print(f"k_sigma = ", cplx_sigma.str_chain())
print()
print("g_*(k_sigma) = ", cplx_g_push.str_chain())
print()
print("h_*(k_sigma) = ", cplx_h_push.str_chain())
print()
print()
print(f"k_disk = ", cplx_disk.str_chain())
print()
print("l_*(k_disk) = ", cplx_l_push.str_chain())

In [8]:
# As an example, let us print the matrix Rg_* eta^0 [the simplices are ordered differently than in the paper]
cplx_g_push.matrices[0].print_matrix(title="Rg_* eta^0")

  Rg_* eta^0    234    124    123    034    014    013
------------  -----  -----  -----  -----  -----  -----
          34      1      0      0      1      0      0
          24      1      1      0      0      0      0
          23      1      0      1      0      0      0
          14      0      1      0      0      1      0
          13      0      0      1      0      0      1
          12      0      1      1      0      0      0
          04      0      0      0      1      1      0
          03      0      0      0      1      0      1
          01      0      0      0      0      1      1
           4      0      0      0      0      0      0


## Discrete morse map and microsupport

We define a map $f: S^2 \rightarrow \mathbb{R}$ and compute ranks of hypercohomology groups as in Figure 10 of the paper

In [16]:
# define filtration of lambd
morse = {
    'A' : {(2,)},
    'B' : {(4,),(2,4)},
    'C' : {(3,),(2,3)},
    'D' : {(1,),(1,2)},
    'E' : {(0,),(0,3)},
    'F' : {(3,4),(2,3,4)},
    'G' : {(0,4),(0,3,4)},
    'H' : {(1,4),(1,2,4)},
    'I' : {(0,1),(0,1,4)},
    'J' : {(1,3),(1,2,3)},
    'K' : {(0,1,3)}
}



# Construct the table

table = []

for name, orig_cplx_name, cplx in (('g', 'k_Sigma', cplx_g_push), ('h', 'k_Sigma', cplx_h_push), ('l', 'k_disk', cplx_l_push)):

    row = [f"RiZ^! R{name}_* {orig_cplx_name}"]
    for lvl, Z in sorted(morse.items()):
        row.append( cplx.proper_pullback(Z).str_hypercohomology() )
    table.append(row)

    row = [f"Ri<^! R{name}_* {orig_cplx_name}"]
    for lvl in sorted(morse):
        Z = set().union(*[morse[lab] for lab in morse if lab<=lvl])
        row.append( cplx.proper_pullback(Z).str_hypercohomology() )
    table.append(row)

    row = [f"Ri>^* R{name}_* {orig_cplx_name}"]
    for lvl in sorted(morse):
        Z = set().union(*[morse[lab] for lab in morse if lab>=lvl])
        row.append( cplx.pullback( ds.PosetMapInclusion(Z, cplx.poset)).str_hypercohomology() )
    table.append(row)

    # =========

    row = [f"RiZ_! RiZ^* R{name}_* {orig_cplx_name}"]
    for lvl, Z in sorted(morse.items()):
        row.append( cplx.pullback( ds.PosetMapInclusion(Z, cplx.poset)).proper_pushforward(cplx.poset).str_hypercohomology() )
    table.append(row)

    row = [f"Ri<*! R{name}_* {orig_cplx_name}"]
    for lvl in sorted(morse):
        Z = set().union(*[morse[lab] for lab in morse if lab<=lvl])
        row.append( cplx.pullback( ds.PosetMapInclusion(Z, cplx.poset)).str_hypercohomology() )
    table.append(row)
    
    table.append([])
    
for row in table:
    for i in range(len(row)):
        if row[i]  == '0,0,0':
            row[i] = '-----'

print(tabulate.tabulate(
    table[:-1], #cut-off the last empty row
    headers=["dim H^p(-)"]+sorted(morse.keys()),
    tablefmt='grid'))

+--------------------------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+
| dim H^p(-)               | A     | B     | C     | D     | E     | F     | G     | H     | I     | J     | K     |
+==========================+=======+=======+=======+=======+=======+=======+=======+=======+=======+=======+=======+
| RiZ^! Rg_* k_Sigma       | 0,0,1 | 0,1,0 | ----- | ----- | ----- | ----- | ----- | ----- | ----- | ----- | 1,0,0 |
+--------------------------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+
| Ri<^! Rg_* k_Sigma       | 0,0,1 | 0,1,1 | 0,1,1 | 0,1,1 | 0,1,1 | 0,1,1 | 0,1,1 | 0,1,1 | 0,1,1 | 0,1,1 | 1,1,1 |
+--------------------------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+
| Ri>^* Rg_* k_Sigma       | 1,1,1 | 1,1,0 | 1,0,0 | 1,0,0 | 1,0,0 | 1,0,0 | 1,0,0 | 1,0,0 | 1,0,0 | 1,0,0 | 1,0,0 |
+--------------------------+-------+-------+-------+-------+----

# Hopf fibration

We triangulate $S^3$, $S^2$, and the Hopf fibration $\textrm{hopf}:S^3\rightarrow S^2$ as described in a paper by Madahar and Arkaria called _A minimal triangulation of the Hopf map and its application_. Then we compute and analyse the pushforward of the constant sheaf on $S^3$ via the $\textrm{hopf}$ map, and observe that it is *not* decomposible as a direct sum of smaller (co)chain complexes, but that it _almost_ splits. This example explicitly illustrates that the Decomposition Theorem for IC sheaves fails for real manifolds.

In [2]:
# Defining the triangulations as described in "A minimal triangulation of the Hopf map and its application"

dV = {
    ('a0','a2','b0'), ('a0','b0','b1'), ('b0', 'b1', 'c1'), ('b1','c1','c2'), ('a2','c1','c2'), ('a0','a2','c2'),
    ('a0','b1','d0'), ('b1','c2','d0'), ('a0','c2','d0')
}
V = { tuple(sorted( trg+('d1',) )) for trg in dV}

dW = {
    ('a0','a1','b1'), ('a1','a2','b1'), ('a2','b1','b2'), ('a2','b0','b2'),
    ('b1','b2','c2'), ('b0','b2','c2'), ('b0','c0','c2'), ('b0','c0','c1'),
    ('a0','c0','c2'), ('a0','c0','c1'), ('a0','a1','c1'), ('a1','a2','c1'),
    
    ('a2','b0','d1'), ('b0','c1','d1'), ('a2','c1','d1'),
    ('a0','b1','d0'), ('b1','c2','d0'), ('a0','c2','d0')
}
W = { tuple(sorted( trg+('d2',) )) for trg in dW}

M = {
    ('a0','b0','c0','c1'), ('a0','b0','b1','c1'), ('a0','a1','b1','c1'),
    ('a1','a2','b1','c1'), ('a2','b1','c1','c2'), ('a2','b1','b2','c2'),
    ('a2','b0','b2','c2'), ('a0','a2','b0','c2'), ('a0','b0','c0','c2')
}

S3 = ds.SimplicialComplex(M|V|W)
S2 = ds.SimplicialComplex({('a','b','c'), ('a','b','d'),('a','c','d'),('b','c','d')})
hopf = ds.SimplicialMap(S3, S2, { ch : ch[0] for ch in {v[0] for v in S3 if len(v)==1}})

In [3]:
# Compute the injective resolution of the constant sheaf on S^3, and push it forward via the hopf map.
k_S3 = ds.ChainComplex.injective_resolution_of_constant_sheaf(S3)
hopf_push = k_S3.pushforward(hopf).minimize()

In [11]:
# We print all matrices, except for the first and the last which are always zero matrices.
for degree, labeled_matrix in sorted(hopf_push.matrices.items())[1:-1]:
    labeled_matrix.print_matrix(title=f"deg:{degree}", form="short", zero_symbol="-")
    print()

deg:0    bcd    acd    abd    abc
-------  -----  -----  -----  -----
bcd      -      -      -      -
acd      -      -      -      -
abd      -      -      -      -
abc      -      -      -      -
cd       1      1      -      -
bd       1      -      1      -
bc       1      -      -      1
ad       -      1      1      -
ac       -      1      -      1
ab       -      -      1      1

deg:1    bcd    acd    abd    abc    cd    bd    bc    ad    ac    ab
-------  -----  -----  -----  -----  ----  ----  ----  ----  ----  ----
cd       1      1      -      -      -     -     -     -     -     -
bd       1      -      1      -      -     -     -     -     -     -
bc       1      -      -      1      -     -     -     -     -     -
ad       -      1      1      -      -     -     -     -     -     -
ac       -      1      -      1      -     -     -     -     -     -
ab       -      -      1      1      -     -     -     -     -     -
d        1      -      -      -      1     1     -   

We see that we can almost split the matrices in blocks, which would represent a split in a direct sum of two (co)chain complexes. The only obstruction is the 1 at position $(d, bcd)$ in degree 1. It is not difficult to see that by allowed row/column operations, we can change the position of this 1 to any place in the lower-left 4x4 block, but we can never get rid of it, because all allowed operations will preserve the parity of 1s in that block. 